In [1]:
#Import required selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Import webdriver_manager for easy management (auto-updation, global path) of driver executable
from webdriver_manager.chrome import ChromeDriverManager

import time #time for various wait/sleep functions
import csv #csv for storing the scraped data

In [20]:
#Import required selenium modules
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Import webdriver_manager for easy management (auto-updation, global path) of driver executable
from webdriver_manager.chrome import ChromeDriverManager

import time #time for various wait/sleep functions
import csv #csv for storing the scraped data
#The link to scrap
url = "https://www.olx.co.id/jakarta-selatan_g4000030/q-mobil"

#Set chromedriver location (automated using webdriver-manager)
service = Service(executable_path=ChromeDriverManager().install())
#create a chrome webdriver instance
driver = webdriver.Chrome(service=service)
#navigate the browser instance to the url
driver.get(url)


while True:
    '''
    While loop to load all results for the search query by finding and clicking on the "Load More" button unless
    the button doesn't exist i.e., all results loaded, no more results to load. Without this, only the first 20 listings
    will be scraped
    '''
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@data-aut-id="btnLoadMore"]')))
        driver.find_element(By.XPATH, '//*[@data-aut-id="btnLoadMore"]').click()
        print("Clicked 'Load More'")
    except ( NoSuchElementException, TimeoutException, StaleElementReferenceException ):
        break
      

'''FIND LISTINGS AND STORE TO A LIST'''
#Finding the UL element containing all the search result items (as list items)
# listings_list = driver.find_element(By.CSS_SELECTOR, "#container > main > div > div > section > div > div > div:nth-child(6) > div._3etsg > div > div:nth-child(2) > ul")
car_listing_ul = None
all_uls = driver.find_elements(By.TAG_NAME, 'ul')
for uls in all_uls:
    if uls.get_attribute("data-aut-id") == "itemsList":
        car_listing_ul = uls

#Finding All list items (li) inside the UL list
listings = car_listing_ul.find_elements(By.TAG_NAME, "li")

#Initialize empty list to store scrapped data (for further writing on a csv file)
scraped_datas = []

#Loop through each list element (list item) and further find required info from their respective XPath
for listing in listings:
    if listing.get_attribute('data-aut-id') == 'itemBox':
        try:
            ad_link = listing.find_element(By.XPATH, "./a").get_attribute("href")
        except NoSuchElementException:
            print("No ad_link")
            break
       
        try:
            ad_price = listing.find_element(By.CSS_SELECTOR, "span[data-aut-id='itemPrice']").text
        except NoSuchElementException:
            print("No ad_price")
        try:
            ad_title = listing.find_element(By.CSS_SELECTOR, "span[data-aut-id='itemTitle']").text
        except NoSuchElementException:
            print("No ad_title")
        try:
            ad_detail = listing.find_element(By.CSS_SELECTOR, "span[data-aut-id='itemDetails']").text
        except NoSuchElementException:
            print("No ad_detail")
        try:
            ad_location = listing.find_element(By.CSS_SELECTOR, "span[data-aut-id='item-location']").text.split("\n")[0]
        except NoSuchElementException:
            print("No ad_location")
            
        scraped_datas.append([ad_link, ad_price, ad_title, ad_detail, ad_location])

'''WRITE SCRAPED LIST TO CSV'''

with open(f'scraped_data/mobil-olx-2.csv', 'w', newline='', encoding="utf-8") as file: 
    writer = csv.writer(file)
    headers = ['Ad Link','Price', 'Ad Title', 'Ad Detail', 'Ad Location']
    writer.writerow(headers)
    for data in scraped_datas:
        writer.writerow(data)

Clicked 'Load More'
Clicked 'Load More'
No ad_detail
No ad_detail
No ad_detail


In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("scraped_data/house-price-bogor-olx.csv")

In [6]:
data

,Ad Link,Price,Ad Title,Ad Detail,Ad Location
0,https://www.olx.co.id/item/dijual-rumah-di-lok...,Rp 595.000.000,Dijual Rumah di lokasi Strategis di kota Bogor,2 KT - 2 KM - 60 m2,"TANAH SEREAL, BOGOR KOTA"
1,https://www.olx.co.id/item/dijual-rumah-murah-...,Rp 2.150.000.000,Dijual rumah murah di bogor bnr,4 KT - 4 KM - 250 m2,"TAMANSARI, BOGOR KAB."
2,https://www.olx.co.id/item/dijual-murah-bu-ruk...,Rp 210.000.000,"Dijual Murah, BU, Ruko Dekat St. Cilebut (200 ...",1 KT - 1 KM - 31 m2,"TANAH SEREAL, BOGOR KOTA"
3,https://www.olx.co.id/item/rumah-strategis-dek...,Rp 777.000.000,Rumah Strategis Dekat Tol & Stasiun di Sindang...,2 KT - 2 KM - 65 m2,"BOGOR BARAT - KOTA, BOGOR KOTA"
4,https://www.olx.co.id/item/dijual-cepat-rumah-...,Rp 650.000.000,Dijual Cepat Rumah Bogor Ciomas Dekat RSUD,3 KT - 2 KM - 150 m2,"BOGOR SELATAN - KOTA, BOGOR KOTA"
...,...,...,...,...,...
135,https://www.olx.co.id/item/jual-murah-rumah-3-...,Rp 249.000.000,Jual Murah Rumah 3 Kamar IPB Bogor,3 KT - 1 KM - 60 m2,"BOGOR BARAT - KOTA, BOGOR KOTA"
136,https://www.olx.co.id/item/bismillah-jual-cepa...,Rp 1.700.000.000,Bismillah jual cepat rumah dalam cluster siap ...,4 KT - 4 KM - 140 m2,"TANAH SEREAL, BOGOR KOTA"
137,https://www.olx.co.id/item/perumahan-islami-di...,Rp 774.000.000,"Perumahan Islami di Cifor Dekat Cilendek, Stas...",3 KT - 2 KM - 60 m2,"BOGOR BARAT - KOTA, BOGOR KOTA"
138,https://www.olx.co.id/item/di-jual-murah-rumah...,Rp 500.000.000,DI JUAL MURAH RUMAH DI VILA CIOMAS INDAH BOGOR,2 KT - 1 KM - 60 m2,"CIOMAS, BOGOR KAB."


In [31]:
data.drop("Bathrooms", inplace=True, axis=1)

In [33]:
data.insert(3,"Bathrooms","")

In [34]:
data.head()

,Ad Link,Price,Bedrooms,Bathrooms,Area,Ad Title,Ad Detail,Ad Location
0,https://www.olx.co.id/item/dijual-rumah-di-lok...,Rp 595.000.000,,,,Dijual Rumah di lokasi Strategis di kota Bogor,2 KT - 2 KM - 60 m2,"TANAH SEREAL, BOGOR KOTA"
1,https://www.olx.co.id/item/dijual-rumah-murah-...,Rp 2.150.000.000,,,,Dijual rumah murah di bogor bnr,4 KT - 4 KM - 250 m2,"TAMANSARI, BOGOR KAB."
2,https://www.olx.co.id/item/dijual-murah-bu-ruk...,Rp 210.000.000,,,,"Dijual Murah, BU, Ruko Dekat St. Cilebut (200 ...",1 KT - 1 KM - 31 m2,"TANAH SEREAL, BOGOR KOTA"
3,https://www.olx.co.id/item/rumah-strategis-dek...,Rp 777.000.000,,,,Rumah Strategis Dekat Tol & Stasiun di Sindang...,2 KT - 2 KM - 65 m2,"BOGOR BARAT - KOTA, BOGOR KOTA"
4,https://www.olx.co.id/item/dijual-cepat-rumah-...,Rp 650.000.000,,,,Dijual Cepat Rumah Bogor Ciomas Dekat RSUD,3 KT - 2 KM - 150 m2,"BOGOR SELATAN - KOTA, BOGOR KOTA"


In [36]:
ad_detail = data['Ad Detail']

In [77]:
len(ad_detail[0])

19

In [76]:
len(ad_detail[4])

20

In [89]:
bedrooms = []

for i in ad_detail:
    bedroom = [i][0]
    
    

In [72]:
bathroom = ad_detail[0][7]
bathroom

'2'

In [74]:
area = ad_detail[4][-6:][:3]
area

'150'